# Projeto Integrador I - EDA

A imobiliária Properati publica periodicamente informações sobre ofertas de imóveis para venda e aluguel. Deve aconselhar o agente imobiliário a desenvolver um modelo de regressão que permita prever o preço por metro quadrado de um imóvel. O objetivo final é que o modelo desenvolvido por eles seja utilizado como avaliador automático a ser aplicado nos próximos imóveis que forem comercializados pela empresa. Para isso, a empresa disponibiliza um conjunto de dados correspondente ao primeiro semestre de 2017.


**Objetivos:**


*   Executar uma limpeza do conjunto de dados fornecido. Em particular, você precisará elaborar estratégias para lidar com dados ausentes em determinadas variáveis.
*   Realizar uma análise descritiva das principais variáveis.
*   Criar novas colunas de determinados recursos que podem ter valor preditivo.


## Importando a Base de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
prop = pd.read_csv(r"C:\Users\RedRagon\Documents\ProjetoIntegrador1\Data\Real_State_Properati.csv",sep='\t')

## Entendo as informações que possuímos na base



### Visualizando a base

In [ ]:
prop.shape

In [ ]:
prop.head(5)

### Verificando se existem valores vazios ou nulos e os tipos das colunas

In [ ]:
prop.isna().sum()

In [ ]:
prop.info()

### Verificando quantos itens distintos possuímos nas colunas

In [ ]:
for col in prop.columns:
    print(col, "-", len(prop[col].unique()))

In [ ]:
prop['property_type'].unique()

### Verificando qual a quantidade e o peso de cada tipo de Propriedade

In [ ]:
prop.groupby(by = 'property_type').count()['place_name'].sort_values(ascending = False)

In [ ]:
prop['property_type'].value_counts()/len(prop['property_type'])*100

### Verificando qual a quantidade e o peso de cada tipo de Moeda


In [ ]:
prop.groupby(by = 'currency').count()['place_name'].sort_values(ascending = False)

In [ ]:
prop['currency'].value_counts()/len(prop['currency'])*100

## Limpando os Dados

### Removendo colunas desnecessárias

Como o país e o estados são os mesmos pra todo o dataset, então podemos retirá-los pois não irá inteferir em nosso estudo.

In [ ]:
prop.drop(columns=['country_name','state_name'], inplace= True)

In [ ]:
prop.columns

### Normalizando o preço com base em dólares e removendo colunas desnecessárias

In [ ]:
prop["price_usd"] = prop.apply(lambda x: x["price"] if x["currency"] == "USD" else x["price"]/117.41,axis=1)

Como deixamos tudo em dólar e ainda criamos outra coluna para esse preço em dólar, podemos então tirar as colunas "price" e "currency" pois não fará diferença em nossos estudos também.

In [ ]:
prop.drop(columns=['price','currency'], inplace= True)

In [ ]:
prop.columns

### Criando uma coluna com o preço por metro quadrado

In [ ]:
prop["preco_metro2"] = prop.price_usd / prop.surface_total_in_m2

In [ ]:
prop.columns

### Verificando a distribuição dos dados para as colunas de tamanho e preço de propriedade

In [ ]:
for i in prop.property_type.unique():
    print(i)
    print(prop.loc[prop.property_type == i,['preco_metro2']].describe())
    print('--------------------')

#### Identificado que existe problema com a coluna preço por metro quadrado para o apartamentos

In [ ]:
for i in prop.property_type.unique():
    print(i)
    print(prop.loc[prop.property_type == i, 'preco_metro2'].std())
    print('--------------------')

Olhando o notebook, verificamos que contém alguns imóveis com a surface igual a 0.

In [ ]:
prop[(prop.property_type == 'apartment') & (prop.surface_total_in_m2 == 0)]

#### Removendo essas linhas e verificando os novos valores de desvio padrão

In [ ]:
prop.drop(index = list(prop[(prop.property_type == 'apartment') & (prop.surface_total_in_m2 == 0)].index),inplace = True)

In [ ]:
for i in prop.property_type.unique():
    print(prop.loc[prop.property_type == i, 'preco_metro2'].std())

### Limpando os dados conforme a distribuição - Via Desvio Padrão

In [ ]:
for i in prop.property_type.unique():
    print(i)
    print("Temos", prop.loc[(prop.property_type == i) & (prop.preco_metro2 > 3*prop.preco_metro2.std()),'preco_metro2'].shape[0], "linha(s) onde o valor supera 3 vezes o desvio padrão")
    print('-------------------------------------------------------------')

In [ ]:
for i in prop.property_type.unique():
    prop.drop(index = list(prop.loc[(prop.property_type == i) & (prop.preco_metro2 > 3*prop.preco_metro2.std())].index), inplace = True)

In [ ]:
for i in prop.property_type.unique():
    print(i)
    print(prop.loc[prop.property_type == i,['preco_metro2']].describe())
    print('--------------------')

In [ ]:
prop.shape

## Plotandos os dados

In [ ]:
ax = sns.violinplot(data = prop, x = "property_type" , y = "preco_metro2", palette='Set2')
plt.title("Distribuição das propriedades por valor")
ax.figure.set_size_inches(20, 5)

In [ ]:
ay = sns.histplot(data=prop, x="preco_metro2", hue="property_type", multiple='stack', palette='Set2')
plt.title("Distribuição das propriedades por valor")
ay.figure.set_size_inches(20, 5)

## Criando variáveis

### Faixas de Preco

In [ ]:
prop['Faixas_de_preco_m2'] = pd.qcut(x = prop.preco_metro2, q = 10, labels = [1,2,3,4,5,6,7,8,9,10])

In [ ]:
prop.head()

In [ ]:
tb = prop.loc[prop.property_type == 'apartment', ['place_name','Faixas_de_preco_m2', 'preco_metro2']].\
pivot_table(values = 'place_name', index = 'place_name', columns = 'Faixas_de_preco_m2', aggfunc = 'count')
tb.round()

In [ ]:
az = sns.heatmap( data = tb.sort_values(by = 10, ascending = False).head(20) , linewidths=1, cmap="YlGnBu")
az.figure.set_size_inches(20, 10)